# Test ONNX Model with Label Matching in C#
This notebook demonstrates how to load a pre-downloaded ONNX model and tokenizer, and match a sample text to predefined labels using cosine similarity.

In [3]:
// Reference the required NuGet package
#r "nuget: Microsoft.ML.OnnxRuntime, 1.21.0"
#r "nuget: Microsoft.ML.Tokenizers, 1.0.2"

Installed Packages Microsoft.ML.OnnxRuntime, 1.21.0 Microsoft.ML.Tokenizers, 1.0.2

In [5]:
// Import required namespaces
using System;
using System.IO;
using System.Linq;
using Microsoft.ML.OnnxRuntime;
using Microsoft.ML.OnnxRuntime.Tensors;
using Microsoft.ML.Tokenizers;

// Define paths to the model and tokenizer
var modelPath = "../artifacts/model/model.onnx";
var vocabPath = "../artifacts/model/vocab.txt";

// Define labels and text
var labels = new string[] { "Shopping", "Exercise", "Hobbies" };
var text = "I need to buy chicken and rolls for dinner.";

// Load ONNX model and tokenizer
var session = new InferenceSession(modelPath);
var tokenizer = BertTokenizer.Create(vocabPath);

// Helper function to tokenize text
IReadOnlyCollection<NamedOnnxValue> Tokenize(string toTokenize)
{
    var inputIds = tokenizer.EncodeToIds(toTokenize);
    var inputIdsTensor = new DenseTensor<long>(inputIds.Select(i => (long)i).ToArray(), new[] { 1, inputIds.Count });

    var typeIds = tokenizer.CreateTokenTypeIdsFromSequences(inputIds);
    var typeIdsTensor = new DenseTensor<long>(typeIds.Take(inputIds.Count).Select(i => (long)i).ToArray(), new[] { 1, inputIds.Count });

    var attentionMask = inputIds.Select(id => id == 0 ? 0L : 1L).ToArray();
    var attentionMaskTensor = new DenseTensor<long>(attentionMask, new[] { 1, inputIds.Count });

    return [
        NamedOnnxValue.CreateFromTensor("input_ids", inputIdsTensor),
        NamedOnnxValue.CreateFromTensor("token_type_ids", typeIdsTensor),
        NamedOnnxValue.CreateFromTensor("attention_mask", attentionMaskTensor)
    ];
}

// Tokenize text and get embeddings
var textTokens = Tokenize(text);
var textEmbedding = session.Run(textTokens)[0].AsTensor<float>();

// Tokenize labels and get embeddings
var labelTokens = labels.Select(Tokenize).ToArray();
var labelEmbeddings = labelTokens.Select(token => session.Run(token)[0].AsTensor<float>()).ToArray();

// Compute cosine similarity
float CosineSimilarity(Tensor<float> a, Tensor<float> b)
{
    var dotProduct = a.Zip(b, (x, y) => x * y).Sum();
    var magnitudeA = (float)MathF.Sqrt(a.Sum(x => x * x));
    var magnitudeB = (float)MathF.Sqrt(b.Sum(x => x * x));
    return dotProduct / (magnitudeA * magnitudeB);
}

// Compute cosine similarity
var similarities = labelEmbeddings.Select(labelEmbedding => CosineSimilarity(textEmbedding, labelEmbedding)).ToArray();

// Find the best matching label
var bestLabelIndex = Array.IndexOf(similarities, similarities.Max());
var bestLabel = labels[bestLabelIndex];
Console.WriteLine($"Sample text: \"{text}\"");
Console.WriteLine($"Matched label: {bestLabel}");